November 3 , 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

docker container start hfpt_Oct28

( docker container id is c9b676310ea0 )

This docker container did not start with the models 'models--meta-llama--Llama-2-7b-hf' or 'models--meta-llama--Llama-2-13b-hf' already downloaded, so what I did was manually copy them from my local '/home/rob/Data2/huggingface/transformers' folder into this container into the folder with the same name.

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
!ls /home/rob/Data2/huggingface/transformers

models--bert-base-cased
models--decapoda-research--llama-13b-hf
models--facebook--bart-large-cnn
models--google--pegasus-cnn_dailymail
models--gpt2
models--gpt2-xl
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--meta-llama--Llama-2-7b-chat-hf
models--meta-llama--Llama-2-7b-hf
models--openai-gpt
models--t5-large
models--transformersbook--pegasus-samsum
models--xlm-roberta-base
tmp560c_s3e
tmpea0zbrrj
tmpni3ccozw
version.txt


Deleting these next 3 models from the container frees up 78gb free - 131gb free = 53gb of space!

I am going to keep the 'models--meta-llama--Llama-2-13b-chat-hf' model and play with this.

In [3]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf

In [4]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf

In [5]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf

In [ ]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf

All of these models load to the 4090 

In [3]:
checkpoint = 'meta-llama/Llama-2-7b'    # Nope!
checkpoint = 'meta-llama/Llama-2-7b-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-7b-chat'    #Nope! 
checkpoint = 'meta-llama/Llama-2-7b-chat-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-13b'     # Nope!
checkpoint = 'meta-llama/Llama-2-13b-hf'  # Works! 14006MiB

checkpoint = 'meta-llama/Llama-2-13b-chat' # Nope!
checkpoint = 'meta-llama/Llama-2-13b-chat-hf' #Works! 14006MiB


In [4]:
# OK! This now works! Sweet! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [5]:
# This works! and is already part of this container! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [6]:
# OK! This now works! Sweet! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# This works! and is already part of this container! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
        )

In [10]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 1. A car is a vehicle that is used to transport people. A truck is a vehicle that is used to transport goods. 2. Cars are typically smaller than trucks and have a higher speed limit. 3. Cars are typically more expensive than trucks. 4. C'}]
CPU times: user 10.6 s, sys: 427 ms, total: 11.1 s
Wall time: 11.3 s


In [11]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
        )

In [12]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 4 wheel drive and 4 wheel steer.\nQuestion: What is the difference between a 2x4 and a 4x4?\nAnswer: One goes in the front, the other goes in the back.\nQuestion: What is the difference between a 4x4 and a 4x2?\nAnswer: One goes in the front, the other goes in the back, and one goes under the front, the other goes under the back.\nQuestion: What is the difference between a 4x4 and a 4x2x4?\nAnswer: One goes in the front, the other goes in the back, and one goes under the front, the other goes under the back, and one goes on top of the other.\nQuestion: What is the difference between a 4x4 and a 4x2x4x4?\nAnswer: One goes in the front, the other goes in the back, and one goes under the front, the other goes under the back, and one goes on top of the other, and one goes in the front, the other goes in the back, and one goes under the front, the other goes under the back, and one goes on top of the o

In [ ]:
# while True:
#     question = input("Question: ")
#     prompt = f"Question: {question}\nAnswer: "
#     result = pipe(prompt)[0]["generated_text"][len(prompt):]
#     print(f"Answer: {result}")

In [14]:
%%time
print(pipe("Question: What is the square root of 25?\nAnswer: "))

[{'generated_text': 'Question: What is the square root of 25?\nAnswer: 5 is the square root of 25.\nQuestion: What is the square root of 26?\nAnswer: 5.13 is the square root of 26.\nQuestion: What is the square root of 27?\nAnswer: 5.22 is the square root of 27.\nQuestion: What is the square root of 28?\nAnswer: 5.31 is the square root of 28.\nQuestion: What is the square root of 29?\nAnswer: 5.40 is the square root of 29.\nQuestion: What is the square root of 30?\nAnswer: 5.50 is the square root of 30.\nQuestion: What is the square root of 31?\nAnswer: 5.60 is the square root of 31.\nQuestion: What is the square root of 32?\nAnswer: 5.70 is the square root of 32.\nQuestion: What is the square root of 33?\nAnswer: 5.80 is the square root of 33.\nQuestion: What is the'}]
CPU times: user 43.4 s, sys: 26 ms, total: 43.5 s
Wall time: 43.5 s
